In [1]:
# --- CELDA 1: LIBRERÍAS Y PREPARACIÓN DE DATOS ---
import os
import shutil

# 1. Instalar librerías
!pip install -q transformers accelerate

# 2. BUSCADOR ESPECÍFICO (Ignora el modelo V6)
print("📂 Buscando dataset_v13_ANATOMY.txt ...")
found_file = None

target_filename = "dataset_v13_ANATOMY.txt" # El nombre exacto que buscamos

for root, dirs, files in os.walk("/kaggle/input"):
    for file in files:
        # Solo nos interesa TU dataset, no los archivos del modelo V6
        if file == target_filename:
            found_file = os.path.join(root, file)
            print(f"   -> ¡ENCONTRADO!: {found_file}")
            break
    if found_file: break

if found_file:
    # 3. TRUCO ANTI-ERROR: Copiar a /kaggle/working/
    # Esto soluciona el error "Read-only file system"
    print("🚚 Moviendo dataset a zona de escritura (/kaggle/working/)...")
    
    writable_path = "/kaggle/working/dataset_training.txt"
    shutil.copy(found_file, writable_path)
    
    dataset_path = writable_path # Usaremos esta ruta a partir de ahora
    print(f"✅ TODO LISTO. Usaremos: {dataset_path}")
    
else:
    print("\n❌ ERROR: No encuentro 'dataset_v13_ANATOMY.txt'.")
    print("   Revisa que el dataset esté subido correctamente en el panel derecho.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 96.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 69.6 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installe

In [2]:
# --- CELDA 2: CARGAR MODELO BASE ---
from transformers import GPT2Tokenizer, GPT2LMHeadModel

model_name = "gpt2" # Usamos el base de OpenAI (Limpio)

print(f"🧠 Descargando {model_name}...")
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Fix para padding (necesario para GPT-2)
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = model.config.eos_token_id

print("✅ Modelo cargado en memoria.")

2025-12-06 18:45:30.941349: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765046731.125897      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765046731.178552      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

✅ Modelo cargado en memoria.


In [3]:
# --- CELDA 3: CIRUGÍA DE TOKENS (ANATOMÍA + MATRIX) ---
print("💉 Preparando inyección de vocabulario...")

# 1. Tokens de Estructura (Matrix)
structure_tokens = [f"<L{i:02d}>" for i in range(1, 100)]
structure_tokens += ["[TOP]", "[MID]", "[BOT]"]
structure_tokens += [f"[S:{i:02d}]" for i in range(65)] # Espacios hasta 64

# 2. Tokens de Anatomía (Master Edition)
anatomy_tokens = [
    "[EYES_COMPLEX]", "[EYES_SIMPLE]", "[EARS]", "[MOUTH/NOSE]", "[ARMS/LEGS]", 
    "[PAWS/FEET]", "[SWORD_BLADE]", "[SWORD_GUARD]", "[SWORD_HANDLE]", 
    "[FLOWER_HEAD]", "[LEAVES/BUSH]", "[TRUNK/STEM]", "[MUSHROOM_CAP]", 
    "[STARS/SPACE]", "[ROCKET_BODY]", "[FLAMES/EXHAUST]", "[PLANET]", "[UFO_DOME]",
    "[ROOF]", "[WALL/BRICK]", "[WINDOW]", "[TOWER_TOP]", "[DOOR/GATE]",
    "[DRAGON_WINGS]", "[SCALES/SKIN]", "[CLAWS]", "[SMOKE/MAGIC]",
    "[WHEEL]", "[WINDSHIELD]", "[BUMPER]", "[TEXTURE_SOLID]", "[TEXTURE_LIGHT]", 
    "[BORDER]", "[GENERIC]", "[WING]", "[SWORD_TIP]", "[SWORD_PART]", "[WHISKERS]", "[TAIL]"
]

# 3. Fusión
all_new_tokens = structure_tokens + anatomy_tokens
num_added = tokenizer.add_tokens(all_new_tokens)
print(f"   -> Se han añadido {num_added} tokens nuevos al vocabulario.")

# 4. Redimensionar el modelo (Agrandar el cerebro)
model.resize_token_embeddings(len(tokenizer))

print("✅ Cirugía completada con éxito.")

💉 Preparando inyección de vocabulario...
   -> Se han añadido 206 tokens nuevos al vocabulario.


The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


✅ Cirugía completada con éxito.


In [4]:
# --- CELDA 4: PROCESAR DATASET ---
from transformers import TextDataset, DataCollatorForLanguageModeling

# Verificamos que la variable exista (por si saltaste la celda 1)
if 'dataset_path' not in globals():
    dataset_path = "/kaggle/working/dataset_training.txt"

print(f"📚 Leyendo dataset desde: {dataset_path}")

try:
    train_dataset = TextDataset(
        tokenizer=tokenizer,
        file_path=dataset_path,
        block_size=256,
        overwrite_cache=True # Forzar re-creación de caché en zona segura
    )
    
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, 
        mlm=False
    )
    print("✅ Dataset tokenizado y listo para entrenar.")
    
except Exception as e:
    print(f"❌ Error procesando dataset: {e}")

📚 Leyendo dataset desde: /kaggle/working/dataset_training.txt


/usr/local/lib/python3.11/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


✅ Dataset tokenizado y listo para entrenar.


In [5]:
# --- CELDA 5: ENTRENAMIENTO ---
from transformers import Trainer, TrainingArguments

# Configuración "Hardcore" para Kaggle (T4 x2)
training_args = TrainingArguments(
    output_dir="./v13_results",
    overwrite_output_dir=True,
    num_train_epochs=5,            # 5 Épocas (suficiente para ver si aprende)
    per_device_train_batch_size=8, # Kaggle aguanta 8 u 16. Prueba 8 primero.
    save_steps=2000,
    save_total_limit=2,
    learning_rate=5e-4,            # Velocidad alta inicial
    prediction_loss_only=True,
    fp16=True,                     # Usar GPU Tensor Cores (Más rápido)
    logging_steps=100,
    report_to="none"               # Desactivar WandB para que no moleste
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)

print("🚀 INICIANDO ENTRENAMIENTO V13...")
trainer.train()

🚀 INICIANDO ENTRENAMIENTO V13...


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
100,1.238300
200,0.919400
300,0.873100
400,0.837800
500,0.809600
600,0.767400
700,0.774800
800,0.739800
900,0.736200
1000,0.728700


We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
You may ignore this warning if your `pad_token_id` (50256) is identical to the `bos_token_id` (50256), `eos_token_id` (50256), or the `sep_token_id` (None), and your input is not padded.


TrainOutput(global_step=5745, training_loss=0.6372228800057327, metrics={'train_runtime': 3708.4409, 'train_samples_per_second': 24.775, 'train_steps_per_second': 1.549, 'total_flos': 1.200310272e+16, 'train_loss': 0.6372228800057327, 'epoch': 5.0})

In [6]:
# --- CELDA 6: GUARDAR MODELO ---
output_path = "./ASCII-GPT-V13-ANATOMIST"

print("💾 Guardando modelo final...")
trainer.save_model(output_path)
tokenizer.save_pretrained(output_path)

print(f"✅ Modelo guardado en {output_path}")
print("   (Recuerda descargar la carpeta o hacer un zip desde el panel derecho)")

💾 Guardando modelo final...
✅ Modelo guardado en ./ASCII-GPT-V13-ANATOMIST
   (Recuerda descargar la carpeta o hacer un zip desde el panel derecho)


In [7]:
# --- CELDA DE SEGURIDAD Y DESCARGA AUTOMÁTICA ---
import shutil
from IPython.display import FileLink

# 1. Comprimir la carpeta del modelo
output_dir = "./ASCII-GPT-V13-ANATOMIST"
zip_name = "MODELO_RESCATADO_V13.zip"

print("📦 Comprimiendo modelo para evacuación inmediata...")
shutil.make_archive(zip_name.replace('.zip', ''), 'zip', output_dir)

print("✅ Compresión lista.")

# 2. Generar Link de Descarga
print(f"⬇️ HAZ CLIC AQUÍ ABAJO PARA DESCARGARLO YA:")
display(FileLink(zip_name))

📦 Comprimiendo modelo para evacuación inmediata...
✅ Compresión lista.
⬇️ HAZ CLIC AQUÍ ABAJO PARA DESCARGARLO YA:


/kaggle/working/MODELO_RESCATADO_V13.zip

In [2]:
import os
import zipfile

# 1. Configuración de Rutas
# Asumimos que el zip está en el directorio actual (si lo acabas de subir)
zip_path = "MODELO_RESCATADO_V13.zip" 
extract_path_v13 = "/kaggle/input/ascii-art-v13/pytorch/default/1"

# 2. Descomprimir V13
if not os.path.exists(extract_path_v13):
    print(f"📦 Descomprimiendo {zip_path}...")
    try:
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_path_v13)
        print("✅ V13 Descomprimido y listo.")
    except FileNotFoundError:
        print(f"❌ ERROR: No encuentro '{zip_path}'. Súbelo a Kaggle primero.")
else:
    print("⚡ V13 ya estaba descomprimido.")

# 3. Instalar librerías necesarias
!pip install -q transformers

⚡ V13 ya estaba descomprimido.


In [3]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch

# --- CARGAR V13 (EL ANATOMISTA) ---
print("🧠 Cargando V13 (Anatomista)...")
tokenizer_v13 = GPT2Tokenizer.from_pretrained(extract_path_v13)
model_v13 = GPT2LMHeadModel.from_pretrained(extract_path_v13)
model_v13.eval() # Modo evaluación

# --- CARGAR V6 (EL ANTIGUO) ---
# BUSCA LA RUTA EN TU PANEL DERECHO (INPUTS) Y PÉGALA AQUÍ
# Debería ser algo como /kaggle/input/ascii-art-v6/...
# Si no sabes la ruta exacta, corre os.listdir("/kaggle/input") en otra celda
path_v6 = "/kaggle/input/ascii-art-v6/pytorch/default/1" # <--- ¡VERIFICA ESTO!

print(f"🧠 Cargando V6 (Old School) desde {path_v6}...")
try:
    tokenizer_v6 = GPT2Tokenizer.from_pretrained(path_v6)
    model_v6 = GPT2LMHeadModel.from_pretrained(path_v6)
    model_v6.eval()
    print("✅ Ambos modelos cargados en CPU.")
except OSError:
    print("❌ ERROR CON V6: Revisa la ruta 'path_v6'. Copiala del panel derecho.")

2025-12-06 20:33:39.647827: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765053219.965968      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765053220.059775      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

🧠 Cargando V13 (Anatomista)...
🧠 Cargando V6 (Old School) desde /kaggle/input/ascii-art-v6/pytorch/default/1...
✅ Ambos modelos cargados en CPU.


In [5]:
import re

def render_v13_vector(prompt_text, temp=0.4):
    # Formato específico del V13
    full_prompt = f"<|startoftext|>\nUser: Dibuja un ASCII art de {prompt_text}\nAI:\n<L01>"
    
    inputs = tokenizer_v13(full_prompt, return_tensors="pt")
    
    # Generar (CPU)
    with torch.no_grad():
        outputs = model_v13.generate(
            **inputs,
            max_new_tokens=350,
            do_sample=True,
            temperature=temp,
            top_k=50,
            top_p=0.95,
            repetition_penalty=1.15, # Evitar bucles
            pad_token_id=tokenizer_v13.eos_token_id,
            eos_token_id=tokenizer_v13.eos_token_id
        )
    
    raw_text = tokenizer_v13.decode(outputs[0], skip_special_tokens=False)
    
    # Limpieza
    if "AI:\n" in raw_text:
        content = raw_text.split("AI:\n")[1]
    else:
        content = raw_text
    content = content.split("<|endoftext|>")[0]

    # --- MOSTRAR RAW (LO QUE PIENSA) ---
    print(f"\n🤖 V13 CÓDIGO (RAW VECTOR):")
    lines = re.split(r'<L\d+>', content)
    for i, line in enumerate(lines[:5]): # Solo primeras 5 líneas para no spamear
        if line.strip(): print(f"<L{i:02d}> {line.strip()}")
    print("... (resto oculto) ...")

    # --- MOSTRAR RENDER (LO QUE DIBUJA) ---
    print(f"\n✨ V13 RENDERIZADO (LIMPIO):")
    final_art = []
    for line in lines:
        if not line.strip(): continue
        
        # 1. Decodificar Espacios [S:xx]
        spaces = 0
        match = re.search(r'\[S:(\d+)\]', line)
        if match: spaces = int(match.group(1))
        
        # 2. Borrar todas las etiquetas [TAG]
        clean_line = re.sub(r'\[.*?\]', '', line).strip()
        
        # 3. Dibujar
        if clean_line:
            final_art.append(" " * spaces + clean_line)
            
    print("\n".join(final_art))
    print("-" * 40)

In [6]:
def battle_arena(prompt, temperatures=[0.2, 0.5, 0.8]):
    print(f"\n{'='*60}")
    print(f"🥊 PELEA DE IA: PROMPT '{prompt}'")
    print(f"{'='*60}")

    for temp in temperatures:
        print(f"\n🌡️ TEMPERATURA: {temp}")
        print(f"{'-'*20}")
        
        # --- TURNO DE V13 (ANATOMISTA) ---
        print("🟦 MODELO V13 (Vectorial):")
        try:
            render_v13_vector(prompt, temp)
        except Exception as e:
            print(f"Error V13: {e}")

        # --- TURNO DE V6 (ANTIGUO) ---
        print("🟧 MODELO V6 (Clásico):")
        full_prompt_v6 = f"User: Dibuja un ASCII art de {prompt}\nAI:\n"
        inputs_v6 = tokenizer_v6(full_prompt_v6, return_tensors="pt")
        
        with torch.no_grad():
            out_v6 = model_v6.generate(
                **inputs_v6,
                max_new_tokens=300,
                do_sample=True,
                temperature=temp,
                repetition_penalty=1.2,
                pad_token_id=tokenizer_v6.eos_token_id
            )
        text_v6 = tokenizer_v6.decode(out_v6[0], skip_special_tokens=True)
        # Limpiar prompt
        clean_v6 = text_v6.replace(full_prompt_v6, "").split("<|endoftext|>")[0]
        print(clean_v6)
        print("-" * 40)

# --- ¡FIGHT! ---
# Probamos las categorías críticas del dataset anatómico
battle_arena("Weapons Swords", temperatures=[0.3, 0.6])
battle_arena("Animals Cats", temperatures=[0.5])
battle_arena("Space Aliens", temperatures=[0.5])


🥊 PELEA DE IA: PROMPT 'Weapons Swords'

🌡️ TEMPERATURA: 0.3
--------------------
🟦 MODELO V13 (Vectorial):

🤖 V13 CÓDIGO (RAW VECTOR):
<L01> [TOP]   [FLAMES/EXHAUST]   [S:05]  ,-""""-.
<L02> [MID]   [TRUNK/STEM]   [S:04]  /        \
<L03> [MID]   [TRUNK/STEM]   [S:03]  |          |
<L04> [BOT]   [MOUTH/NOSE]   [S:03]  \_________/
... (resto oculto) ...

✨ V13 RENDERIZADO (LIMPIO):
     ,-""""-.
    /        \
   |          |
   \_________/
jgs '---------'
     '----'
^jgs^^^^^^^
        '
^jgs^^^^^^^
         '
          '
           '
           '
^jgs^^^^^^^
          '
           '
           '
           '
           '
           '
           '
           '
           '
           '
           '
           '
           '
           '
           '
           '
            '
'
'
----------------------------------------
🟧 MODELO V6 (Clásico):
                                                                                                                                               

In [7]:
import torch
import re

def probe_v13(category, specific_temps=[0.2, 0.5, 0.8]):
    print(f"\n{'='*60}")
    print(f"🔬 SONDA PROFUNDA: {category.upper()}")
    print(f"{'='*60}")
    
    # Prompt forzado V13
    prompt = f"<|startoftext|>\nUser: Dibuja un ASCII art de {category}\nAI:\n<L01>"
    inputs = tokenizer_v13(prompt, return_tensors="pt")
    
    for temp in specific_temps:
        print(f"\n🌡️ TEMPERATURA: {temp}")
        
        with torch.no_grad():
            outputs = model_v13.generate(
                **inputs,
                max_new_tokens=400,
                do_sample=True,
                temperature=temp,
                top_k=50,
                top_p=0.95,
                repetition_penalty=1.12, 
                pad_token_id=tokenizer_v13.eos_token_id,
                eos_token_id=tokenizer_v13.eos_token_id
            )
        
        raw_text = tokenizer_v13.decode(outputs[0], skip_special_tokens=False)
        
        # Extracción
        if "AI:\n" in raw_text:
            content = raw_text.split("AI:\n")[1]
        else:
            content = raw_text
        content = content.split("<|endoftext|>")[0]
        
        # --- ANÁLISIS DE ETIQUETAS (¿Qué está pensando?) ---
        # Buscamos las etiquetas semánticas que usó (ej: [EYES], [ROOF])
        tags_found = set(re.findall(r'\[[A-Z_]+\]', content))
        # Filtramos las aburridas
        tags_found = {t for t in tags_found if t not in ["[TOP]", "[MID]", "[BOT]"]}
        
        print(f"🧠 ETIQUETAS DETECTADAS: {list(tags_found)}")
        
        # --- RENDERIZADO ---
        print("-" * 40)
        lines = re.split(r'<L\d+>', content)
        for line in lines:
            line = line.strip()
            if not line: continue
            
            # Decodificar espacio [S:xx]
            spaces = 0
            match = re.search(r'\[S:(\d+)\]', line)
            if match: spaces = int(match.group(1))
            
            # Limpiar basura visual
            clean = re.sub(r'\[.*?\]', '', line).strip()
            
            # Filtro anti-basura simple
            if "=====" in clean or "-----" in clean or len(clean) > 80:
                continue
                
            print(" " * spaces + clean)
        print("-" * 40)

# --- BATERÍA DE PRUEBAS ---

# 1. Castillos (Prueba de alineación vertical y torres)
probe_v13("Buildings Castles", specific_temps=[0.3, 0.6])

# 2. Dragones (Prueba de anatomía compleja: alas, garras)
probe_v13("Fantasy Dragons", specific_temps=[0.5, 0.7])

# 3. Insectos (Prueba de patas y simetría pequeña)
probe_v13("Animals Insects", specific_temps=[0.4])

# 4. Coches (Prueba de ruedas [WHEEL])
probe_v13("Vehicles Cars", specific_temps=[0.4])


🔬 SONDA PROFUNDA: BUILDINGS CASTLES

🌡️ TEMPERATURA: 0.3
🧠 ETIQUETAS DETECTADAS: []
----------------------------------------
|
----------------------------------------

🌡️ TEMPERATURA: 0.6
🧠 ETIQUETAS DETECTADAS: []
----------------------------------------
     ______________
    /______________\
   | |             | |
   |_|___________|_|
ejm97 |_____|
       (___)
        -K-
          `
ejm97      hjw
             `
             nn
               nn
              nn
              nn
              nn
              nn
                nn
                 nn
                  nn
                  nn
                   nn
                    nn
                      nn
                     nn
                      nn
                      nn
                      nn
                      nn
nn
                      n
                      nn
                      nn
                      nn
                      nn
                      n
                      n
                      nn